In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import time
import shutil
import torch.nn as nn
from skimage import io
import torchvision
import cv2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch import ToTensorV2
from torchvision import utils
from torchvision.ops import box_iou
from tqdm import tqdm
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

WEIGHTS_FILE1 = "../input/reef-fasterrcnn-weights/fasterrcnn_resnet50_fpn-e11.bin"
WEIGHTS_FILE1 = "../input/reef-fasterrcnn-weights/fasterrcnn_resnet50_fpn-e10.bin"

In [ ]:
# dasdas

In [ ]:
!ls ../input/reef-fasterrcnn-weights

In [ ]:
def get_model():
    # load a model; pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    num_classes = 2  # 1 class (starfish) + background

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    model.to(DEVICE)
    return model

model = get_model()

In [ ]:
# from typing import List

# def calculate_score(
#     preds: List[torch.Tensor],
#     gts: List[torch.Tensor],
#     iou_th: float
# ) -> float:
#     num_tp = 0
#     num_fp = 0
#     num_fn = 0
#     for p, gt in zip(preds, gts):
#         if len(p) and len(gt):
#             iou_matrix = box_iou(p, gt)
#             tp = len(torch.where(iou_matrix.max(0)[0] >= iou_th)[0])
#             fp = len(p) - tp
#             fn = len(torch.where(iou_matrix.max(0)[0] < iou_th)[0])
#             num_tp += tp
#             num_fp += fp
#             num_fn += fn
#         elif len(p) == 0 and len(gt):
#             num_fn += len(gt)
#         elif len(p) and len(gt) == 0:
#             num_fp += len(p)
#     if (5 * num_tp + 4 * num_fn + num_fp )!=0:
#         score = 5 * num_tp / (5 * num_tp + 4 * num_fn + num_fp )
#     else:
#         score = np.nan
#     if (num_tp+num_fn) != 0:
#         recall = num_tp/ (num_tp+num_fn)
#     else:
#         recall=np.nan
#     if (num_tp+num_fp)!=0:
#         precission = num_tp/ (num_tp+num_fp)
#     else:
#         precission=np.nan


#     return score, precission, recall




In [ ]:
# test_img = np.load("../input/tensorflow-great-barrier-reef/example_test.npy")
# test_img = test_img.reshape((3,3,720,1280))
# test_imgs = [torch.from_numpy(i)/255.0 for i in test_img]

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
BASE_DIR = "../input/tensorflow-great-barrier-reef/train_images/"
# WORKING_DIR = "../input/data-conversion-to-4k/train_img"
WORKING_DIR = "../input/tensorflow-great-barrier-reef/train_images/"
NUM_EPOCHS = 5
train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
# train_df['annotations'].iloc[3]
train_df['annotations'] = train_df['annotations'].apply(eval)
func = lambda x: "video_"+x.split("-")[0]+"/"+x.split("-")[1]+".jpg"
# vid_func = lambda x: "video_"+x.split("-")[0]

train_df["img_path"] = train_df["image_id"].apply(func)
train_df["no_of_bbox"] = train_df["annotations"].apply(lambda x: len(x))

train_df = train_df[train_df["no_of_bbox"]>0].reset_index(drop=True)

In [ ]:
class ReefDataset:

    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
#         self.working_dir = "./train_img/video_0"
        
        
    def can_augment(self, boxes):
        box_outside_image = ((boxes[:, 0] < 0).any() or (boxes[:, 1] < 0).any() 
                             or (boxes[:, 2] > 1280).any() or (boxes[:, 3] > 720).any())
        return not box_outside_image

    def __getitem__(self, i):

        row = self.df.iloc[i]
#         print(WORKING_DIR)
#         print(row["img_path"])
        image = cv2.imread(f'{WORKING_DIR}/{row["img_path"]}').astype(np.float32)
#         print(image.shape)
#         print(f'{BASE_DIR}/{row["img_path"]}')
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /=255.0
        
        
        boxes = pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(np.float32).values
        
        # Change from [x_min, y_min, w, h] to [x_min, y_min, x_max, y_max]
#         boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
#         boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        boxes[:, 2] = np.clip(boxes[:, 0] + boxes[:, 2],0,1280)
        boxes[:, 3] = np.clip(boxes[:, 1] + boxes[:, 3],0,720)
        
        n_boxes = boxes.shape[0]
        
        # Calculate the area
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'area': torch.as_tensor(area, dtype=torch.float32),
            
            'image_id': torch.tensor([i]),
            
            
            'labels': torch.ones((n_boxes,), dtype=torch.int64),
            
            
            'iscrowd': torch.zeros((n_boxes,), dtype=torch.int64)            
        }

        if self.transforms and self.can_augment(boxes):
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': target['labels']
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            if n_boxes > 0:
                target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        else:
            image = ToTensorV2(p=1.0)(image=image)['image']

        return image, target

    def __len__(self):
        return len(self.df)

In [ ]:
df_train = train_df.iloc[0: 4427]
df_val = train_df.iloc[4428:4918]

ds_train = ReefDataset(df_train)
ds_val = ReefDataset(df_val)

In [ ]:
4918 - 4428 

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

dl_train = DataLoader(ds_train, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn)
dl_val = DataLoader(ds_val, batch_size=1, shuffle=False, num_workers=4, collate_fn=collate_fn)

In [ ]:
# def evaluate_f2(confthre):
#     scores = []
#     prec05 = []
#     rec05 = []
#     prec03 = []
#     rec03 = []
#     iou_ths = np.arange(0.3, 0.85, 0.05)
#     with torch.no_grad():
#         for images, targets in tqdm(dl_val):
#             model.eval()
#             images = list(image.to(device) for image in images)
#             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#             preds = model(images)

#             for i in range(len(images)):
#                 preds[i]['boxes']=preds[i]['boxes'].int()
#                 preds[i]['boxes']=preds[i]['boxes'][preds[i]['scores']>confthre]
#                 score = [calculate_score(preds[i]['boxes'].unsqueeze(0), targets[i]['boxes'].unsqueeze(0), iou_th)[0] for iou_th in iou_ths]
#                 scores.append(np.nanmean(score))
#                 prec05.append(calculate_score(preds[i]['boxes'].unsqueeze(0), targets[i]['boxes'].unsqueeze(0), 0.5)[1]) 
#                 prec03.append(calculate_score(preds[i]['boxes'].unsqueeze(0), targets[i]['boxes'].unsqueeze(0), 0.3)[1]) 
#                 rec05.append(calculate_score(preds[i]['boxes'].unsqueeze(0), targets[i]['boxes'].unsqueeze(0), 0.5)[2]) 
#                 rec03.append(calculate_score(preds[i]['boxes'].unsqueeze(0), targets[i]['boxes'].unsqueeze(0), 0.3)[2]) 
#     print(f'F2 Score for confthre , {confthre}, :  {np.nanmean(scores):.3f} Precission .5: {np.nanmean(prec05):.3f} Precission .3: {np.nanmean(prec03):.3f}  Recall .5: {np.nanmean(rec05):.3f} Recall .3: {np.nanmean(rec03):.3f}')

In [ ]:
# evaluate_f2(0.13) # previous conf = 0.66

In [ ]:
print((5 * 0.902 * 0.153) / (4 * 0.902 + 0.153))

In [ ]:
(0.1773944444444444 + 0.18346982185588936)/2

In [ ]:
# qpwijdpjqd qwopidhpiqw eifhpiehwfwe wepfhpiehwfnpiwef oewfpiw
# oafpiahspif aoishfoiasf aoishfiopas
# hpohpihpihpi

In [ ]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
#     print(f"calc_iou shape: {bboxes1.shape}")
#     print(f"bbox: {bboxes1}")
    
#     print(f"calc_iou shape2: {bboxes2.shape}")
#     print(f"bbox2: {bboxes2}")
    
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp)

def calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()
#     print(gt_bboxes)
#     print(pred_bboxes)
#     print(f"shape {pred_bboxes.shape}")
    
    
    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
#         print("calc_is_correct_at_iou_th TYPE: ",type(pred_bbox))
        ious = calc_iou(gt_bboxes, pred_bbox.reshape((1,4)))
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    return tp, fp, fn

def calc_is_correct(gt_bboxes, pred_bboxes, iou_th=0.5):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, 0
        return tps, fps, fns

    elif len(gt_bboxes) == 0:
        tps, fps, fns = 0, len(pred_bboxes), 0
        return tps, fps, fns

    elif len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, len(gt_bboxes)
        return tps, fps, fns

#     pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf

    tps, fps, fns = 0, 0, 0
    tp, fp, fn = calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th)
    tps += tp
    fps += fp
    fns += fn
    return tps, fps, fns

def calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    f2s = []
    for iou_th in np.arange(0.3, 0.85, 0.05):
        tps, fps, fns = 0, 0, 0
        for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
            tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes, iou_th)
            tps += tp
            fps += fp
            fns += fn
            if verbose:
                num_gt = len(gt_bboxes)
                num_pred = len(pred_bboxes)
                print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
        f2 = f_beta(tps, fps, fns, beta=2)    
        print(f'f2@{iou_th}:{f2}')
        f2s.append(f2)
    return np.mean(f2s)

In [ ]:
# ejrfpwoej[forwoerfx]

In [ ]:
# calc_f2_score(gt_list,pred_list)

In [ ]:
# ds_val[12][0].shape

In [ ]:
# model.eval()
# with torch.no_grad():
# #     model = model.to(device)
#     prediction = model([ds_val[12][0].to(DEVICE)])
#     print(prediction)

In [ ]:
preds = []
for images, targets in tqdm(dl_val):
    model.eval()
    images = list(image.to(DEVICE) for image in images)
    targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
#     img = images[0].mul(255).permute(1,2,0).cpu().numpy()
#     plt.imshow(img)
#     print(img.shape)
    with torch.no_grad():
        outputs = model(images)
#     print(outputs)
#     break
    preds.append(outputs)

In [ ]:
# import json
# with open("preds.txt",'wb') as f:
#     f.write(json.dumps(preds))

In [ ]:
targets = []
for image, target in tqdm(dl_val):
    targets.append(target)


# boxes1[:, 2] = boxes1[:, 2] - boxes1[:, 0]
# boxes1[:, 3] = boxes1[:, 3] - boxes1[:, 1]
# print(len(targets))

In [ ]:
# pred_list[0]

In [ ]:
# list(map(int, preds[0][0]["boxes"].tolist()[0]))
# preds[0][0]["boxes"].tolist()[0]

In [ ]:
# lambda x: [round(i) for i in x]
# new_preds = [i.apply(x) for i in preds[0][0]["boxes"].tolist()]
pred_list = []
for j in range(490):
    temp_list1 = []
    for i in preds[j][0]["boxes"].tolist():
        lst = [round(j) for j in i]
        temp_list1.append(lst)
    pred_list.append(np.array(temp_list1))
    
    
# pred_list = []
# for j in range(490):
#     temp_list1 = np.array([])
#     for i in preds[j][0]["boxes"].tolist():
#         lst = np.array([round(j) for j in i])
#         np.append(temp_list1,lst)
#     pred_list.append(temp_list1)
# pred_list

In [ ]:
# pred_list[0]

In [ ]:
# pred_list

In [ ]:
# targets[0][i]["boxes"].tolist()

In [ ]:
# preds[0][0]["boxes"].tolist()

In [ ]:
# gt_list = []
# for j in range(490):
#     empty_list2 = np.array([])
#     for i in targets[j][0]["boxes"].tolist():
#         gtlst = np.array([round(j) for j in i])
# #         print(gtlst)
#         empty_list2 = np.append(empty_list2,gtlst)
#     gt_list.append(empty_list2)

gt_list = []
for j in range(490):
    temp_list2 = []
    for i in targets[j][0]["boxes"].tolist():
        gtlst = [round(j) for j in i]
        temp_list2.append(gtlst)
    gt_list.append(np.array(temp_list2))

# print(type(gt_list[0]))
# print(type(gt_list[0][0]))
len(gt_list[0].shape)

In [ ]:
# len(gt_list[0].shape)

In [ ]:
# empty_array = np. array([])
# to_append = np. array([1, 2, 3])
# combined_array = np.append(empty_array, to_append)
# combined_array

In [ ]:
# gt_list

In [ ]:
for gts in gt_list: 
    boxes1 = gts
    boxes1[:, 2] = boxes1[:, 2] - boxes1[:, 0]
    boxes1[:, 3] = boxes1[:, 3] - boxes1[:, 1]
    gts = boxes1

In [ ]:
for prd in pred_list: 
    boxes2 = prd
    boxes2[:, 2] = boxes2[:, 2] - boxes2[:, 0]
    boxes2[:, 3] = boxes2[:, 3] - boxes2[:, 1]
    prd = boxes2

In [ ]:
calc_f2_score(gt_list,pred_list)

In [ ]:
# plt.imshow(next(iter(dl_val))[0][0].permute(1,2,0).cpu().numpy())

In [ ]:
# preds

In [ ]:
# with torch.no_grad():
# #             outputs = model([img])[0]

In [ ]:
# preds[0]